In [2]:
import pandas as pd
import numpy as np
import math
import datetime

In [4]:
train = pd.read_csv('../data/train_users_2.csv')
test = pd.read_csv('../data/test_users.csv')
countries = pd.read_csv('../data/countries.csv')
user_demo = pd.read_csv('../data/age_gender_bkts.csv')
sessions = pd.read_csv('../data/sessions.csv')
#changing into datetime
train.date_account_created = pd.to_datetime(train.date_account_created)
train.timestamp_first_active = pd.to_datetime(train.timestamp_first_active, format = "%Y%m%d%H%M%S")
train.date_first_booking = pd.to_datetime(train.date_first_booking)
test.timestamp_first_active = pd.to_datetime(test.timestamp_first_active, format = "%Y%m%d%H%M%S")
test.date_account_created = pd.to_datetime(test.date_account_created)

train_destination = train.iloc[:,-1]

In [5]:
unknowntrain = [i for i, j in enumerate(train.gender) if j == '-unknown-']
train.loc[unknowntrain, 'gender'] = 'NA'
unknowntest = [i for i, j in enumerate(test.gender) if j == '-unknown-']
test.loc[unknowntest, 'gender'] = 'NA'

#unknowntest = test.gender.index('-unknown-')
#test.loc[unknowntest, 'gender'] = 'NA'

In [6]:
np.unique(train.gender)

array(['FEMALE', 'MALE', 'NA', 'OTHER'], dtype=object)

In [7]:
#sessions grouping by user

#Group by user_id, aggregate by number of counts (counting device_type as it is never NA), 
#and total sum of elapsed time in seconds
group_sessions = sessions.groupby("user_id").agg({'device_type':'count', 'secs_elapsed':'sum'})
#rename columns
group_sessions.columns = ['sum_secs_elapsed', 'counts']
#group by variable turns into index, I'm reseting the index and putting user_id back as a column
group_sessions.reset_index(level=0, inplace=True)

In [8]:
group_sessions.head()

,user_id,sum_secs_elapsed,counts
0,00023iyk9l,867896.0,40
1,0010k6l0om,586543.0,63
2,001wyh0pz8,282965.0,90
3,0028jgx1x1,297010.0,31
4,002qnbzfs5,6487080.0,789


In [9]:
#bucket all ages into format that user_demo is in for age
def agebuckets(ages):
    ageless =  [i for i in range(5,101,5)] # 5, 10, 15, 20...95, 100
    buckets = ['%d-%d' %(i, i+4) for i in range(0,100,5)] # 0-4, 5-9, 10-14...90-94, 95-99
    newlist = []
    for i in range(len(ages)):
        if math.isnan(ages[i]):
            newlist.append('NA')
        elif ages[i] <ageless[0]:
            newlist.append(buckets[0])
        elif ages[i] < ageless[1]:
            newlist.append(buckets[1])
        elif ages[i] < ageless[2]:
            newlist.append(buckets[2])
        elif ages[i] < ageless[3]:
            newlist.append(buckets[3])
        elif ages[i] < ageless[4]:
            newlist.append(buckets[4])
        elif ages[i] < ageless[5]:
            newlist.append(buckets[5])
        elif ages[i] < ageless[6]:
            newlist.append(buckets[6])
        elif ages[i] < ageless[7]:
            newlist.append(buckets[7])
        elif ages[i] < ageless[8]:
            newlist.append(buckets[8])
        elif ages[i] < ageless[9]:
            newlist.append(buckets[9])
        elif ages[i] < ageless[10]:
            newlist.append(buckets[10])
        elif ages[i] < ageless[11]:
            newlist.append(buckets[11])
        elif ages[i] < ageless[12]:
            newlist.append(buckets[12]) 
        elif ages[i] < ageless[13]:
            newlist.append(buckets[13]) 
        elif ages[i] < ageless[14]:
            newlist.append(buckets[14])
        elif ages[i] < ageless[15]:
            newlist.append(buckets[15])
        elif ages[i] < ageless[16]:
            newlist.append(buckets[16])
        elif ages[i] < ageless[17]:
            newlist.append(buckets[17])
        elif ages[i] < ageless[18]:
            newlist.append(buckets[18])
        elif ages[i] < ageless[19]:
            newlist.append(buckets[19]) 
        else:
            newlist.append('100+')
    return newlist

In [10]:
train.age[0:20]

0      NaN
1     38.0
2     56.0
3     42.0
4     41.0
5      NaN
6     46.0
7     47.0
8     50.0
9     46.0
10    36.0
11    47.0
12     NaN
13    37.0
14    36.0
15    33.0
16     NaN
17    31.0
18     NaN
19    29.0
Name: age, dtype: float64

In [11]:
train.age = agebuckets(train.age)
test.age = agebuckets(test.age)

In [12]:
def timedif(L1, L2):
    timediflist = []
    for i in range(len(L1)):
        try:
            if (L1[i]-L2[i]).days <= -1:#datetime.timedelta(days=0):
                timediflist.append('before')
            elif (L1[i]-L2[i]).days ==0: #datetime.timedelta(days=1):
                timediflist.append('same day')
            else:
                timediflist.append('greater 1 day')
        except:
            timediflist.append('NB')
            
    return timediflist

In [13]:
#adding time lag columns
train['lag_account_created'] = timedif(train.date_first_booking, train.date_account_created)
train['lag_first_active'] = timedif(train.date_first_booking, train.timestamp_first_active)
train['lag_account_created_first_active'] = timedif(train.date_account_created, train.timestamp_first_active)
test['lag_account_created_first_active'] = timedif(test.date_account_created, test.timestamp_first_active)

In [30]:
def bookings(L1, L2, L3, L4):
    timediflist = []
    for i in range(len(L1)):
        if L1[i] == 'same day' or L2[i] == 'same day':
            timediflist.append('early')
        elif L1[i] == 'before' and L2[i] == 'before' and L3[i] == 'same day':
            timediflist.append('early')
        elif L1[i] == 'greater 1 day' and L2[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L1[i] == 'greater 1 day' and L2[i] == 'before':
            timediflist.append('waited')
        elif L1[i] == 'before' and L2[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L1[i] == 'before' and L2[i] == 'before' and L3[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L4[i] == 'NDF':
            timediflist.append('NB')
        else:
            timediflist.append('NA')

            
    return timediflist

In [31]:
booking = bookings(train.lag_account_created, train.lag_first_active, train.lag_account_created_first_active, train.country_destination)

In [32]:
train['bookings'] = booking

In [34]:
#given the train data gender, age, and country_desination produce the corresponding population in thousands
population_in_thous = []
for i in range(train.shape[0]):
    if train.country_destination[i] == 'NDF':
        population_in_thous.append('NB')    
    elif train.gender[i] == 'NA' or train.age[i] == 'NA' or train.gender[i] == 'nan': 
        population_in_thous.append('NA')
    elif train.gender[i] == 'OTHER':
        population_in_thous.append(0)  
    elif train.country_destination[i] == 'other':
        gendersi = user_demo.loc[user_demo.gender == train.gender[i].lower(),:] 
        ages = gendersi.loc[gendersi.age_bucket == train.age[i], :]
        ages = list(map(lambda x: float(x), ages.population_in_thousands))
        population_in_thous.append(np.mean(ages))
    else:
        genders = user_demo.loc[user_demo.gender == train.gender[i].lower(),:] 
        dests = genders.loc[genders.country_destination == train.country_destination[i] ,:]    
        population_in_thous.append(float((dests.loc[dests.age_bucket == train.age[i], 'population_in_thousands'])))
        
population_in_thous[0:10]

['NB',
 'NB',
 11264.0,
 2458.8000000000002,
 'NA',
 'NA',
 10659.0,
 10659.0,
 11413.0,
 'NA']

In [35]:
#merging gender age bucket with train data
train['population_in_thousands'] = population_in_thous

In [36]:
#merging with grouped sessions and countries, **note most of training data is not in sessions. see below 
test_m = pd.merge(test, group_sessions, left_on='id', right_on ='user_id', how='left')
train_m = pd.merge(train, group_sessions, left_on='id', right_on ='user_id', how='left')
test_m = test_m.drop('user_id', 1)
train_m = train_m.drop('user_id', 1)
print train_m.iloc[0:5, 0:10] #to be deleted?
print train_m.iloc[0:5, 10:]  # to be deleted?

           id date_account_created timestamp_first_active date_first_booking  \
0  gxn3p5htnn           2010-06-28    2009-03-19 04:32:55                NaT   
1  820tgsjxq7           2011-05-25    2009-05-23 17:48:09                NaT   
2  4ft3gnwmtx           2010-09-28    2009-06-09 23:12:47         2010-08-02   
3  bjjt8pjhuk           2011-12-05    2009-10-31 06:01:29         2012-09-08   
4  87mebub9p4           2010-09-14    2009-12-08 06:11:05         2010-02-18   

   gender    age signup_method  signup_flow language affiliate_channel  
0      NA     NA      facebook            0       en            direct  
1    MALE  35-39      facebook            0       en               seo  
2  FEMALE  55-59         basic            3       en            direct  
3  FEMALE  40-44      facebook            0       en            direct  
4      NA  40-44         basic            0       en            direct  
  affiliate_provider first_affiliate_tracked signup_app first_device_type  \
0   

In [37]:
train_m.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_device_type,first_browser,country_destination,lag_account_created,lag_first_active,lag_account_created_first_active,bookings,population_in_thousands,sum_secs_elapsed,counts
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NA,NA,facebook,0,en,direct,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,35-39,facebook,0,en,seo,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,55-59,basic,3,en,direct,...,Windows Desktop,IE,US,before,greater 1 day,greater 1 day,waited,11264,NaN,NaN
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,40-44,facebook,0,en,direct,...,Mac Desktop,Firefox,other,greater 1 day,greater 1 day,greater 1 day,waited,2458.8,NaN,NaN
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NA,40-44,basic,0,en,direct,...,Mac Desktop,Chrome,US,before,greater 1 day,greater 1 day,waited,NA,NaN,NaN


In [38]:
train_m.columns

Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination',
       u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'bookings',
       u'population_in_thousands', u'sum_secs_elapsed', u'counts'],
      dtype='object')

In [39]:
train_m.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_device_type,first_browser,country_destination,lag_account_created,lag_first_active,lag_account_created_first_active,bookings,population_in_thousands,sum_secs_elapsed,counts
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NA,NA,facebook,0,en,direct,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,35-39,facebook,0,en,seo,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,55-59,basic,3,en,direct,...,Windows Desktop,IE,US,before,greater 1 day,greater 1 day,waited,11264,NaN,NaN
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,40-44,facebook,0,en,direct,...,Mac Desktop,Firefox,other,greater 1 day,greater 1 day,greater 1 day,waited,2458.8,NaN,NaN
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NA,40-44,basic,0,en,direct,...,Mac Desktop,Chrome,US,before,greater 1 day,greater 1 day,waited,NA,NaN,NaN


In [183]:
#remove id, time format columns and categorical columns
excl = list(train_m.columns)
toremove = ['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 
              'country_destination'] 
map(lambda x: excl.remove(x), toremove)

[None, None, None, None, None]

In [184]:
train_m.loc[:, excl]

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,lag_account_created,lag_first_active,lag_account_created_first_active,bookings,population_in_thousands,sum_secs_elapsed,counts
0,NA,NA,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NB,NB,greater 1 day,NB,NB,NaN,NaN
1,MALE,35-39,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NB,NB,greater 1 day,NB,NB,NaN,NaN
2,FEMALE,55-59,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,before,greater 1 day,greater 1 day,waited,11264,NaN,NaN
3,FEMALE,40-44,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,greater 1 day,greater 1 day,greater 1 day,waited,2458.8,NaN,NaN
4,NA,40-44,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,before,greater 1 day,greater 1 day,waited,NA,NaN,NaN
5,NA,NA,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,greater 1 day,same day,before,early,NA,NaN,NaN
6,FEMALE,45-49,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,greater 1 day,greater 1 day,before,waited,10659,NaN,NaN
7,FEMALE,45-49,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,greater 1 day,greater 1 day,before,waited,10659,NaN,NaN
8,FEMALE,50-54,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,greater 1 day,greater 1 day,before,waited,11413,NaN,NaN
9,NA,45-49,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,same day,before,before,early,NA,NaN,NaN


In [185]:
train_x = train_m.loc[:, excl]
test_x = test_m.loc[:, excl]

In [186]:
test_x

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,lag_account_created,lag_first_active,lag_account_created_first_active,bookings,population_in_thousands,sum_secs_elapsed,counts
0,FEMALE,35-39,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,NaN,NaN,before,NaN,NaN,119187.0,8.0
1,NA,NA,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,NaN,NaN,before,NaN,NaN,250119.0,19.0
2,NA,NA,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NaN,NaN,before,NaN,NaN,975575.0,58.0
3,NA,NA,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE,NaN,NaN,before,NaN,NaN,123001.0,11.0
4,NA,NA,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,NaN,NaN,before,NaN,NaN,454023.0,19.0
5,FEMALE,25-29,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome,NaN,NaN,before,NaN,NaN,60924.0,8.0
6,MALE,45-49,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,NaN,NaN,before,NaN,NaN,640922.0,30.0
7,NA,NA,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NaN,NaN,before,NaN,NaN,391231.0,33.0
8,NA,NA,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NaN,NaN,before,NaN,NaN,8481847.0,283.0
9,NA,NA,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Firefox,NaN,NaN,before,NaN,NaN,5084265.0,115.0


In [187]:
from sklearn.feature_extraction import DictVectorizer
train_x = train_x.T.to_dict().values()
test_x = test_x.T.to_dict().values()

vec = DictVectorizer()
train_x = vec.fit_transform(train_x)
test_x = vec.fit_transform(test_x)

In [201]:
train_y = train_m.loc[:,'country_destination']
test_y = test_m

In [202]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
print type(train_y)

<type 'numpy.ndarray'>


In [197]:
#from sklearn import preprocessing
#le = preprocessing.OneHotEncoder()
#le.fit(train_y)
#train_y = le.transform(train_y)
#print type(train_y)

/Users/Zi/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Zi/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


<class 'scipy.sparse.csr.csr_matrix'>


In [198]:
set(train_y)

{<1x213451 sparse matrix of type '<type 'numpy.float64'>'
 	with 213451 stored elements in Compressed Sparse Row format>}

In [204]:
import xgboost as xgb
import numpy as np
params = {}
params["objective"] = "multi:softmax"
params["num_class"] = 12
params["eta"] = 0.005
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 1
params["silent"] = 1
params["max_depth"] = 6
params['eval_metric'] = "ndcg@5"
params['nthread'] = 4

plst = list(params.items())

dtrain = xgb.DMatrix(train_x, label=train_y)
dtest = xgb.DMatrix(test_x)
num_round = 5
model = xgb.train(plst, dtrain, num_round)
pred = model.predict(dtest)

In [205]:
pred = map(int,pred)

In [206]:
pred = le.inverse_transform(pred)

In [207]:
print set(pred)

set(['US', 'NDF'])


In [208]:
pred

array(['US', 'US', 'NDF', ..., 'NDF', 'US', 'US'], dtype=object)

In [209]:
model.get_fscore()

{'f1': 7,
 'f10': 6,
 'f100': 4,
 'f112': 4,
 'f113': 2,
 'f114': 20,
 'f115': 2,
 'f117': 14,
 'f118': 10,
 'f119': 2,
 'f120': 14,
 'f121': 4,
 'f122': 7,
 'f123': 7,
 'f124': 3,
 'f126': 14,
 'f127': 1,
 'f131': 5,
 'f133': 7,
 'f134': 4,
 'f138': 1,
 'f14': 2,
 'f140': 4,
 'f141': 3,
 'f143': 3,
 'f148': 1,
 'f150': 1,
 'f153': 2,
 'f155': 1,
 'f16': 12,
 'f160': 406,
 'f161': 25,
 'f162': 1,
 'f164': 2,
 'f165': 12,
 'f166': 5,
 'f167': 35,
 'f168': 5,
 'f169': 2,
 'f170': 1,
 'f171': 65,
 'f2': 8,
 'f21': 3,
 'f22': 2,
 'f27': 1,
 'f28': 10,
 'f29': 18,
 'f3': 18,
 'f30': 21,
 'f31': 23,
 'f32': 8,
 'f33': 19,
 'f34': 20,
 'f36': 17,
 'f37': 9,
 'f38': 9,
 'f39': 16,
 'f4': 5,
 'f40': 16,
 'f41': 1,
 'f46': 22,
 'f47': 27,
 'f48': 13,
 'f49': 7,
 'f5': 4,
 'f50': 110,
 'f51': 4,
 'f52': 5,
 'f55': 14,
 'f56': 2,
 'f57': 1,
 'f58': 7,
 'f59': 1,
 'f6': 15,
 'f67': 16,
 'f68': 1,
 'f7': 1,
 'f76': 7,
 'f80': 8,
 'f88': 1,
 'f9': 4}